#  Workarounds for the problem using grid equivalents
This tutorial gives some suggestions to solve the difficulties that arise when using grid equivalent.
In this tutorial, we still use the grid case9 as the test bed:

- internal area (buses): [0, 3, 4, 8]
- boundary buses: [4, 8] (boundary buses belong to the internal area)
- external area (buses): [1, 2, 5, 6, 7] 

<img src="pics/grid areas.png" alt="ALT">

In [1]:
from pandapower.networks.power_system_test_cases import case9
from pandapower.grid_equivalents import get_equivalent
from pandapower.create import create_dcline
from pandapower.run import runpp
from copy import deepcopy

net = case9()

## A) dc line

If the to-be-reduced grid has dc-lines. We need convert them before starting grid equivalent. First of all, let's create a dc-line. In the following, the line (index=2) between bus 4 and bus 8 is replaced by a dc-line

In [2]:
net.line.drop([2],inplace=True)
create_dcline(net, 4, 5, 50, 0.5, 0.7, 1.0, 1.0)
runpp(net)

Now, we have got a grid with dc-line (see A)).
<img src="pics/grid areas (workarounds).png" alt="ALT">

using *_add_dcline_gens* the dc-lines are represented by generators. Then, we need to remote or disconnect the origin dc_line or close them. 

In [3]:
from pandapower.auxiliary import _add_dcline_gens
net2 = deepcopy(net)
_add_dcline_gens(net2)
net2.dcline.in_service=False
runpp(net2)

We can compare the power flow results between *net1* and *net2* and carry out the grid equivalent process.

In [4]:
print("net with dcline", net.res_bus)
print("net with represented dcline", net2.res_bus)
net_eq = get_equivalent(net2, "rei", [4,8], [0])
print("equivalent net", net_eq.res_bus)

net with dcline       vm_pu  va_degree        p_mw     q_mvar
0  1.000000   0.000000  -87.837523 -30.233733
1  1.000000  26.074443 -163.000000 -39.113351
2  1.000000  32.268568  -85.000000  -2.118240
3  0.983887  -2.947620    0.000000   0.000000
4  1.000000 -11.008845   90.000000  30.000000
5  1.000000  29.413484    0.000000   0.000000
6  0.971830  21.570967  100.000000  35.000000
7  0.980859  20.112778    0.000000   0.000000
8  0.926759   0.514121  125.000000  50.000000
net with represented dcline       vm_pu  va_degree        p_mw     q_mvar
0  1.000000   0.000000  -87.837523 -30.233733
1  1.000000  26.074443 -163.000000 -39.113351
2  1.000000  32.268568  -85.000000  -2.118240
3  0.983887  -2.947620    0.000000   0.000000
4  1.000000 -11.008845  140.000000 -46.050989
5  1.000000  29.413484  -49.050000 -12.806786
6  0.971830  21.570967  100.000000  35.000000
7  0.980859  20.112778    0.000000   0.000000
8  0.926759   0.514121  125.000000  50.000000
equivalent net        vm_pu  va_degr

## B) slack in the external area

In some cases, there are slack buses in the external area, see B). It could cause power flow problems during the grid equivalent calculation. In this case, we can consider the slack buses as the boundary to reserve them.
In the grid *case9*, there are two generators in the external area. We activate their slack function and treat the corresponding buses as the boundary.

In [5]:
net = case9()
net.gen.slack = True
runpp(net)
boundary_buses = [4, 8, 1, 2]
net_eq = get_equivalent(net, "rei", [4,8], [0])
print("original net", net.res_bus)
print("equivalent net", net_eq.res_bus)

original net       vm_pu  va_degree        p_mw     q_mvar
0  1.000000   0.000000 -127.525028 -21.454707
1  1.000000   0.000000 -100.862955  -6.607394
2  1.000000   0.000000  -89.753317   4.804554
3  0.990370  -4.253458    0.000000   0.000000
4  0.979249  -6.930476   90.000000  30.000000
5  1.004194  -3.002286    0.000000   0.000000
6  0.987125  -5.749658  100.000000  35.000000
7  0.997864  -3.622033    0.000000   0.000000
8  0.963808  -8.068145  125.000000  50.000000
equivalent net        vm_pu  va_degree        p_mw     q_mvar
0   1.000000   0.000000 -127.525028 -21.454707
3   0.990370  -4.253458    0.000000   0.000000
4   0.979249  -6.930476   89.278340   8.055702
8   0.963808  -8.068145  124.576477  32.191499
9   0.987125  -5.749658   99.315369   0.532046
10  1.000000   0.000000 -100.055429 -16.812750
11  1.000000   0.000000  -88.619419 -10.155545
